In [ ]:
# Load all of the relevant packages

# For communicating with the Keysight SMU; Installation: pip install pyvisa
import pyvisa

# For keeping track of time
import time


# For maniputalting data; Installation: pip install numpy
import numpy as np


# For maniputalting data; Installation: pip install pandas
import pandas as pd

# Matplotlib for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation, rc, cm
import matplotlib.colors as colors

# SciPy for image processing; Installation: pip install scipy
from scipy import ndimage
from scipy import signal
import scipy.signal
import scipy.misc
from scipy.interpolate import griddata
from scipy.optimize import fmin_powell
import scipy.ndimage as snd
from scipy.fft import fft, ifft, fftfreq

# tkinter for interactive path generation
from tkinter import filedialog
from tkinter import *

import pickle
